In [1]:
import cv2
print(cv2.__version__)
import os
print("当前工作目录:", os.getcwd())



4.6.0
当前工作目录: d:\vscodeproject\soft_fingers\Weld_sin_Use_UR_with_rotation\ControlCode_Use_UR_Rewrite_sin_rotation


In [7]:
for i in range(0, 8):
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        print( 'camera open')
        while True:
            ret, img = cap.read()
            print(ret)
            if not ret:
                continue
            else:
                print(ret, "camera id:")
                print(img.shape)
            #

                cv2.imshow('5', img)
                cv2.waitKey(10)
                c = cv2.waitKey(1000)
                if c == 27:
                        break
                cap.set(cv2.CAP_PROP_FPS, 330)
                cv2.destroyAllWindows()
    cap.release()
    cv2.destroyAllWindows()
    # break

camera open
True
True camera id:
(480, 640, 3)
True
True camera id:
(480, 640, 3)
camera open
True
True camera id:
(480, 640, 3)
True
True camera id:
(480, 640, 3)
True
True camera id:
(480, 640, 3)
True
True camera id:
(480, 640, 3)
True
True camera id:
(480, 640, 3)
True
True camera id:
(480, 640, 3)
True
True camera id:
(480, 640, 3)
True
True camera id:
(480, 640, 3)
True
True camera id:
(480, 640, 3)
True
True camera id:
(480, 640, 3)
True
True camera id:
(480, 640, 3)
True
True camera id:
(480, 640, 3)
camera open
True
True camera id:
(480, 640, 3)
True
True camera id:
(480, 640, 3)
True
True camera id:
(480, 640, 3)


In [8]:
from cv2 import aruco
import cv2
import cv2.aruco as aruco

In [9]:
# 创建 4X4_100 字典
aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_250)

# 生成一个 ID 为 42 的 ArUco 标记
marker_id = 30
marker_size = 200  # 标记的尺寸（以像素为单位）
marker_image = aruco.drawMarker(aruco_dict, marker_id, marker_size)


In [10]:
cv2.imshow('ArUco Marker', marker_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [1]:
# print(aruco_dict)
# print(aruco.DICT_4X4_100)
import cv2, os, yaml
import pandas as pd
config = yaml.load(open('./controller_config.yaml'), Loader=yaml.FullLoader)

In [2]:
import numpy as np
import time
from queue import LifoQueue, Full

class GetFingerForce:
    def __init__(self, tag_pose_vecs):
        self.p0 = np.mean(tag_pose_vecs[:, :], axis=0)
        # 取均值
        self.dpLast = np.zeros([6])
        self.dp = np.zeros([12])
        self.ImgQue = LifoQueue(100)
        # 后进先出队列，最新的图像会被最先处理
        self.StopImgRead = LifoQueue(1000)

        self.Pose = np.zeros(0)
        self.LastTime = time.time()

    def add_image(self, img):
        try:
            self.ImgQue.put(img, block=False)
        except Full:
            print("队列已满，无法添加新的图像")

# 示例用法
tag_pose_vecs = np.random.rand(10, 3)  # 示例数据
get_finger_force = GetFingerForce(tag_pose_vecs)

# 尝试添加一些图像到队列中
for _ in range(105):  # 尝试添加超过队列容量的图像
    img = np.random.rand(640, 480, 3)  # 示例图像数据
    get_finger_force.add_image(img)

print("ImgQue size:", get_finger_force.ImgQue.qsize())
print("程序未崩溃，继续运行")

队列已满，无法添加新的图像
队列已满，无法添加新的图像
队列已满，无法添加新的图像
队列已满，无法添加新的图像
队列已满，无法添加新的图像
ImgQue size: 100
程序未崩溃，继续运行


In [3]:
from scipy.spatial.transform import Rotation as Rt
def rpy2rot_vec(rpy):
    r = Rt.from_euler('xyz', rpy)
    rot_vec = r.as_rotvec()
    return rot_vec
    # 欧拉角转换成旋转向量
tcp = rpy2rot_vec((-np.pi, 0, 0))
print(tcp)

[-3.14159265  0.          0.        ]


In [4]:
import numpy as np
import cv2
import cv2.aruco as aruco
parameters = aruco.DetectorParameters_create()
aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_100)
# camera_matrix =np.matrix ([[ 522.1905267183170736, 0.0000000000000000, 308.8532705794233948  ],
#                   [ 0.0000000000000000, 522.8166355728753842, 234.2130798407917496 ],
#                   [ 0.0000000000000000, 0.0000000000000000, 1.0000000000000000 ]])
# # camera_matrix = [[1000, 0, 320], [0, 1000, 240], [0, 0, 1]]  # 示例相机矩阵
# camera_dist = np.matrix([ 0.1058497356483027 ,-0.1488874492570700 ,0.0000000000000000 ,0.0000000000000000 ,-0.8698561798572152])

camera_matrix = np.matrix( [[3.20790520e+03, 0.00000000e+00, 1.01091874e+03],
                  [0.00000000e+00, 3.39787065e+03, 6.44844641e+02],
                  [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
camera_dist = np.matrix( [ 3.40400374e-01, -2.65574509e+00,  9.96946780e-03,  8.68103031e-03,  -4.46459652e+01])
  # marker_length: 0.012
marker_length = 0.006
parameters

< cv2.aruco.DetectorParameters 000001A3B6960490>

In [6]:
import cv2
import time
import cv2.aruco as aruco
from scipy.spatial.transform import Rotation as R

# 创建视频捕捉对象，0 表示使用默认摄像头
cap = cv2.VideoCapture(1)

# 创建 ArUco 检测参数
parameters = aruco.DetectorParameters_create()

# 加载预定义的 ArUco 字典
# aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_250)
aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_250)
i = 0
while True:
    # 读取一帧图像
    time.sleep(1)
    ret, frame = cap.read()
    if not ret:
        break

    # 将图像转换为灰度图
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 检测 ArUco 标记
    corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
    rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners,
                                                          marker_length,
                                                          camera_matrix,
                                                          camera_dist)
    
    # 估计标记的姿态
    rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners, marker_length, camera_matrix, camera_dist)

    # 检查返回值
    if rvecs is not None and len(rvecs) > 0:
        print("Rotation Vectors:", rvecs)
        # Rotation Vectors: [[[-3.12682862  0.03934772  0.00940102]]]
        print("Translation Vectors:", tvecs)
        # Translation Vectors: [[[0.00055413 0.00014394 0.02618797]]]
        
        # 处理第一个标记的旋转向量
        rr = R.from_rotvec(rvecs[0][0])
        print("Rotation Matrix:\n", rr.as_matrix())
         # [[ 0.99966529 -0.02520588 -0.00582934]
         # [-0.02511868 -0.9995782   0.01457627]
         # [-0.00619429 -0.01442496 -0.99987677]]
    
        # 获取欧拉角（以度为单位）
        rpy = rr.as_euler('xyz', degrees=True)

    
        # 打印欧拉角
        print("Euler Angles (before adjustment):", rpy)
        # Euler Angles (before adjustment): [-179.17346597    0.35490888   -1.4393736 ]
    
        # 确保 rpy 是一个数组并且长度为 3
        if rpy.shape == (3,):
            rpy[-1] = rpy[-1] + 360 * (rpy[-1] < 0)
            rpy[0] = rpy[0] + 360 * (rpy[0] < 0)
            print("Euler Angles (after adjustment):", rpy)
            # Euler Angles (after adjustment): [1.80826534e+02 3.54908876e-01 3.58560626e+02]
            
            # 检查 tvecs[0][0] 是否有效
            if tvecs is not None and len(tvecs) > 0:
                print("Translation Vector (in mm):", tvecs[0][0] * 1000)
                # 将平移向量转换为毫米并合并
                back = np.concatenate((tvecs[0][0] * 1000, rpy))
                print("Final Vector:", back)
 #                Final Vector: [5.54131605e-01 1.43942161e-01 2.61879730e+01 1.80826534e+02
 # 3.54908876e-01 3.58560626e+02]
            else:
                print("Translation vector is None or empty.")
        else:
            print("Unexpected shape for Euler angles:", rpy.shape)
    else:
        print("未检测到任何标记或估计姿态失败")


    # 如果检测到标记，绘制边框并显示 ID
    if ids is not None:
        frame = aruco.drawDetectedMarkers(frame, corners, ids)
        cv2.drawFrameAxes(frame, camera_matrix, camera_dist, rvecs[i], tvecs[i], 0.1)
        # print('corners:',i, corners)

    # 显示结果图像
    cv2.imshow('Aruco Detection', frame)

    # 按下 'q' 键退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 释放视频捕捉对象并关闭所有窗口
cap.release()
cv2.destroyAllWindows()

未检测到任何标记或估计姿态失败
Rotation Vectors: [[[-2.61793836 -0.52675947 -0.26916079]]]
Translation Vectors: [[[-0.02209879 -0.01064906  0.10285197]]]
Rotation Matrix:
 [[ 0.90784566  0.40748497  0.09885471]
 [ 0.31886342 -0.82401649  0.46831928]
 [ 0.27229098 -0.39364047 -0.87801185]]
Euler Angles (before adjustment): [-155.8517981   -15.80063906   19.35284908]
Euler Angles (after adjustment): [204.1482019  -15.80063906  19.35284908]
Translation Vector (in mm): [-22.09879034 -10.64905928 102.85196542]
Final Vector: [-22.09879034 -10.64905928 102.85196542 204.1482019  -15.80063906
  19.35284908]
Rotation Vectors: [[[-2.59728519 -0.51507435 -0.25669207]]]
Translation Vectors: [[[-0.02126291 -0.01210094  0.10312447]]]
Rotation Matrix:
 [[ 0.91172108  0.40125735  0.08807498]
 [ 0.31191775 -0.8156716   0.48722393]
 [ 0.26734244 -0.41674018 -0.86882428]]
Euler Angles (before adjustment): [-154.37483288  -15.50618765   18.88682823]
Euler Angles (after adjustment): [205.62516712 -15.50618765  18.88682823